In [44]:
import pandas as pd
import sklearn as sk

In [45]:
ruta = 'archiverotten/rotten_tomatoes_critic_reviews.csv'
df = pd.read_csv(ruta)

In [46]:
df.head(5)

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [47]:
#Filtrado de columnas que se van a utilizar
columnas = ['review_score','review_content']
df = df[columnas]
df.head(5)

,review_score,review_content
0,NaN,A fantasy adventure that fuses Greek mythology...
1,NaN,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,NaN,With a top-notch cast and dazzling special eff...
3,3.5/5,Whether audiences will get behind The Lightnin...
4,NaN,What's really lacking in The Lightning Thief i...


In [48]:
df.dropna(inplace=True)

In [49]:
# Diccionario de conversión de letras a escala de 5 como fracción
letter_to_score = {
    'A': '5/5',
    'A-': '4.7/5',
    'B+': '4.3/5',
    'B': '4/5',
    'B-': '3.7/5',
    'C+': '3.3/5',
    'C': '3/5',
    'C-': '2.7/5',
    'D+': '2.3/5',
    'D': '2/5',
    'D-': '1.7/5',
    'F': '1/5',
}

# Función para reemplazar calificaciones de letra
def convert_score(score):
    score_str = str(score).strip()
    if score_str in letter_to_score:
        return letter_to_score[score_str]
    return score_str  # deja igual si ya es una fracción como "3.5/5"

# Aplica la función a la columna de calificaciones
df['review_score'] = df['review_score'].apply(convert_score)

In [52]:
df.head(5)

,review_score,review_content
3,3.5/5,Whether audiences will get behind The Lightnin...
6,1/4,Harry Potter knockoffs don't come more transpa...
7,3.5/5,"Percy Jackson isn't a great movie, but it's a ..."
8,4/5,"Fun, brisk and imaginative"
9,3/5,"Crammed with dragons, set-destroying fights an..."


In [54]:
# Convertir fracciones a número decimal
def fraction_to_float(score_str):
    try:
        num, den = score_str.split('/')
        return float(num) / float(den)
    except:
        return None  # para manejar errores o valores inesperados

df['review_score'] = df['review_score'].apply(fraction_to_float)


In [56]:
df.head(5)

,review_score,review_content
3,0.70,Whether audiences will get behind The Lightnin...
6,0.25,Harry Potter knockoffs don't come more transpa...
7,0.70,"Percy Jackson isn't a great movie, but it's a ..."
8,0.80,"Fun, brisk and imaginative"
9,0.60,"Crammed with dragons, set-destroying fights an..."


In [58]:
# Calcular la media
mean_score = df['review_score'].mean()
print("Media de review_score:", round(mean_score, 3))

Media de review_score: 0.651


In [67]:
# Clasificar como 'positivo' si el score es mayor que la media, si no, 'negativo'
df['sentiment'] = df['review_score'].apply(
    lambda x: 'positivo' if x >= mean_score else 'negativo'
)

In [68]:
df

,review_score,review_content,sentiment
3,0.70,Whether audiences will get behind The Lightnin...,positivo
6,0.25,Harry Potter knockoffs don't come more transpa...,negativo
7,0.70,"Percy Jackson isn't a great movie, but it's a ...",positivo
8,0.80,"Fun, brisk and imaginative",positivo
9,0.60,"Crammed with dragons, set-destroying fights an...",negativo
...,...,...,...
1130006,0.80,As a spectacular war film with a powerful mora...,positivo
1130013,0.70,"Seen today, it's not only a startling indictme...",positivo
1130014,0.86,A rousing visual spectacle that's a prequel of...,positivo
1130015,0.70,"A simple two-act story: Prelude to war, and th...",positivo


In [69]:
print(df['sentiment'].value_counts())

sentiment
positivo    386806
negativo    371903
Name: count, dtype: int64
